In [1]:
%load_ext autoreload

In [2]:
import os
import gzip
import pandas as pd
from os.path import join, basename

In [3]:
# from utils import indir
# from utils import outdir
# from utils import skiprows
# from utils import genome_ids
# from utils import metacols
# from utils import cols

In [4]:
indir = "/home/lhvu/bio/1000-vcf/vcf/"
outdir = "/home/lhvu/bio/1000-vcf/vcf/parsed"

metacols = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']
genome_ids = ['HG02070', 'HG01849', 'HG01874', 'HG01599', 'HG01872', 'HG02121', 'HG02078', 'HG02128', 'HG02087', 'HG01846', 'HG02113', 'HG02088', 'HG02140', 'HG02086', 'HG02023', 'HG01843', 'HG02081', 'HG01865', 'HG01597', 'HG01873', 'HG02085', 'HG02139', 'HG01847', 'HG02073', 'HG01848', 'HG02079', 'HG02133', 'HG02028', 'HG01869', 'HG02035', 'HG02137', 'HG02029', 'HG01842', 'HG02138', 'HG02116', 'HG02084', 'HG01867', 'HG02025', 'HG01844', 'HG01850', 'HG02136', 'HG02075', 'HG01857', 'HG01855', 'HG02016', 'HG01866', 'HG02522', 'HG02048', 'HG02026', 'HG02130', 'HG01861', 'HG01596', 'HG02047', 'HG01841', 'HG02060', 'HG01863', 'HG02134', 'HG02067', 'HG02020', 'HG01878', 'HG02072', 'HG01864', 'HG01870', 'HG02512', 'HG02069', 'HG01871', 'HG02141', 'HG02082', 'HG01860', 'HG02049', 'HG01859', 'HG01845', 'HG02142', 'HG02122', 'HG01868', 'HG02019', 'HG01858', 'HG02127', 'HG02032', 'HG02017', 'HG02513', 'HG02057', 'HG02040', 'HG02521', 'HG02050', 'HG02031', 'HG01853', 'HG02061', 'HG01840', 'HG02131', 'HG01851', 'HG02058', 'HG01595', 'HG01598', 'HG02064', 'HG01600', 'HG01862', 'HG01852', 'HG02076']
cols = metacols + genome_ids

skiprows = [i for i in range(19)]

# Input file. Note: this need to change for each chromosome

In [5]:
filename = "ALL.chr1.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf"

In [6]:
infile = join(indir, filename) 

In [7]:

outfile = join(outdir, f"{filename}.filtered.csv")


In [8]:
len(genome_ids)

99

# Read the file from disk, will take some time. Note, we read the uncompressed file

In [9]:

df = pd.read_csv(infile, sep = "\t", skiprows=skiprows, usecols=cols)


# After reading the file, hre is the quick check

In [10]:
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG01595,...,HG02137,HG02138,HG02139,HG02140,HG02141,HG02142,HG02512,HG02513,HG02521,HG02522
0,1,10416,.,CCCTAA,C,.,PASS,AC=240;AN=5096;DP=365460;AF=0.05;EAS_AF=0.06;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,1,16103,.,T,G,.,PASS,AC=118;AN=5096;DP=29994;AF=0.02;EAS_AF=0;EUR_A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,1,17496,.,AC,A,.,PASS,AC=25;AN=5096;DP=189765;AF=0;EAS_AF=0;EUR_AF=0...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,1,51479,.,T,A,.,PASS,AC=531;AN=5096;DP=17461;AF=0.1;EAS_AF=0;EUR_AF...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1
4,1,51898,.,C,A,.,PASS,AC=426;AN=5096;DP=15331;AF=0.08;EAS_AF=0.05;EU...,GT,0|0,...,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [11]:
df.shape

(6191833, 108)

# Now, create a mask for all rows that have the above genome_ids that are equal to the target value '0|0'

In [12]:
target_value = '0|0'

In [13]:
mask = (df[genome_ids] == target_value).all(axis=1)

In [14]:
mask[0:10]

0    False
1     True
2     True
3    False
4    False
5     True
6     True
7    False
8     True
9    False
dtype: bool

In [15]:
matching_rows = df[mask]

In [16]:
matching_rows.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG01595,...,HG02137,HG02138,HG02139,HG02140,HG02141,HG02142,HG02512,HG02513,HG02521,HG02522
1,1,16103,.,T,G,.,PASS,AC=118;AN=5096;DP=29994;AF=0.02;EAS_AF=0;EUR_A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,1,17496,.,AC,A,.,PASS,AC=25;AN=5096;DP=189765;AF=0;EAS_AF=0;EUR_AF=0...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
5,1,51928,.,G,A,.,PASS,AC=367;AN=5096;DP=17022;AF=0.07;EAS_AF=0.01;EU...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
6,1,51954,.,G,C,.,PASS,AC=1;AN=5096;DP=18469;AF=0;EAS_AF=0;EUR_AF=0;A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
8,1,54669,.,C,T,.,PASS,AC=1;AN=5096;DP=23532;AF=0;EAS_AF=0;EUR_AF=0;A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


# We are interested in non-matching rows

In [17]:
non_matching_rows = df[~mask]

In [18]:
non_matching_rows.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG01595,...,HG02137,HG02138,HG02139,HG02140,HG02141,HG02142,HG02512,HG02513,HG02521,HG02522
0,1,10416,.,CCCTAA,C,.,PASS,AC=240;AN=5096;DP=365460;AF=0.05;EAS_AF=0.06;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,1,51479,.,T,A,.,PASS,AC=531;AN=5096;DP=17461;AF=0.1;EAS_AF=0;EUR_AF...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1
4,1,51898,.,C,A,.,PASS,AC=426;AN=5096;DP=15331;AF=0.08;EAS_AF=0.05;EU...,GT,0|0,...,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
7,1,54490,.,G,A,.,PASS,AC=494;AN=5096;DP=24006;AF=0.1;EAS_AF=0;EUR_AF...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1
9,1,54708,.,G,C,.,PASS,AC=1198;AN=5096;DP=23694;AF=0.24;EAS_AF=0.07;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


# Save file to disk

In [19]:
outfile = join(outdir, outfile)

In [20]:
non_matching_rows.to_csv(outfile, index=False)

# Save a copy in .gz to disk for portability

In [21]:
outfile_gz = join(outdir, f"{filename}.filtered.csv.gz")

In [22]:
non_matching_rows.to_csv(outfile_gz, index=False)